In [1]:
import pymeshlab
import numpy as np

In [2]:
ms = pymeshlab.MeshSet()

ms.load_new_mesh("model_normalized.obj")

m = ms.current_mesh()

v_matrix = m.vertex_matrix()
f_matrix = m.face_matrix()

By default faces are defined and stored in the counter-clockwise order, so should not have any issues calculating the moments

In [3]:
def C_ijk(C,i,j,k):
    return (C[0]**i)*(C[1]**j)*(C[2]**k)*np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))
        

def D_ijk(D_tensor,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # D_ijk=0
        pass
    elif (0==i) and (0==j) and (0==k):
        # D_ijk = 1
        D_tensor[i,j,k] = 1
    else:
        D_tensor[i,j,k] = B[0]*D_tensor[i-1,j,k]+B[1]*D_tensor[i,j-1,k]+B[2]*D_tensor[i,j,k-1]+C_ijk(C,i,j,k)

def S_ijk(S_tensor,D_tensor,A,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # S_ijk = 0
        pass
    elif (0==i) and (0==j) and (0==k):
        # S_ijk = 1
        S_tensor[i,j,k] = 1
    else:
        S_tensor[i,j,k] = A[0]*S_tensor[i-1,j,k]+A[1]*S_tensor[i,j-1,k]+A[2]*S_tensor[i,j,k-1]+D_tensor[i,j,k]
    
        
def vol(A,B,C):
    return 1./6*np.linalg.det([A,B,C])


def M_ijk(f_matrix, v_matrix, max_m):
    M_tensor = np.zeros([max_m,max_m,max_m])
    
    
    for face_num in range(len(f_matrix)):
        
        
        # Get the three coordinates A,B,C
        # Any mesh library should return these in CCW order
        [A,B,C] = v_matrix[f_matrix[face_num]]
        
        # Calculate the face moments tensor
        # Each face moments tensor requires calculating S_ijk, D_ijk, and C_ijk Tensor Calculations.

        # C_ijk tensor is not reused, calculate on the fly
        
        # D_ijk tensor is reused, calculate and store
        D_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        D_ijk(D_tensor,B,C,i,j,k)
                        
        # S_ijk tensor is reqused, calculate and store
        S_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        S_ijk(S_tensor,D_tensor,A,B,C,i,j,k)
                        
        # Now calculate the moments on current face
        f_M_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        f_M_tensor[i,j,k] = (np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3)*np.linalg.det([A,B,C])*S_tensor[i,j,k]
        
        # Add contribution to the total moments
        M_tensor = M_tensor+f_M_tensor
                
    return M_tensor



This is the final loss function that gives the scalar.

In [4]:
def mesh_moment_loss(f_matrix_1, v_matrix_1, f_matrix_2, v_matrix_2, max_m = 5):
    return np.linalg.norm(M_ijk(f_matrix_1, v_matrix_1,max_m)-M_ijk(f_matrix_1, v_matrix_1,max_m))

Note that this loss function is directly comparing the moments.

Matching the moment series will make two shapes match each other.

However, if we want to make the loss function invariant to scale, translation, and rotation, (or any symmetries) then the invariants of the loss function can be calcuated from the moments tensor we have just calculated.
Matching the invariants will make two shapes "match" each other without being affected by scale, translation, and rotation.

This is easy to implement, will do this next.

When two objects are the exact same, the loss function is supposed to be 0

In [5]:
mesh_moment_loss(f_matrix,v_matrix,f_matrix,v_matrix,max_m=5)

0.0